In [1]:
import pandas as pd

motor_file = '/Users/larsheijnen/Thesis/data/motor/MDS-UPDRS_Part_III_21Mar2025.csv'
mri_file = '/Users/larsheijnen/Thesis/data/MRI/Magnetic_Resonance_Imaging__MRI__30Mar2025.csv'
output_file = '/Users/larsheijnen/Thesis/data/MRI/merged_motor_mri_data_by_patno_event.csv'

In [2]:
motor_df = pd.read_csv(motor_file)
print(motor_df.shape)

mri_df = pd.read_csv(mri_file)
print(mri_df.shape)

(32346, 63)
(7596, 15)


In [3]:
merge_cols = ['PATNO', 'EVENT_ID']
missing_motor_cols = [col for col in merge_cols if col not in motor_df.columns]
missing_mri_cols = [col for col in merge_cols if col not in mri_df.columns]

if missing_motor_cols:
    print(f"\nError: Merge columns missing in motor_df: {missing_motor_cols}")
    exit()
if missing_mri_cols:
    print(f"\nError: Merge columns missing in mri_df: {missing_mri_cols}")
    exit()

In [4]:
date_formats_to_try = ["%m/%Y", "%m/%d/%Y", "%Y-%m-%d %H:%M:%S.%f", "%Y-%m-%d"] # Add other expected formats

def robust_date_parse(series, formats):
    """Tries multiple formats to parse dates, returns NaT on failure."""
    parsed_series = pd.NaT
    for fmt in formats:
        try:
            # Try parsing with the current format, only update NaT values
            current_parsed = pd.to_datetime(series, format=fmt, errors='coerce')
            if parsed_series is pd.NaT:
                 parsed_series = current_parsed
            else:
                parsed_series = parsed_series.fillna(current_parsed)
        except (ValueError, TypeError):
             # Ignore if format doesn't match at all for the series type
            continue
    # Final attempt with automatic inference for any remaining NaTs
    if parsed_series is pd.NaT: # Handle case where series was completely unparseable initially
        parsed_series = pd.to_datetime(series, errors='coerce')
    else:
         parsed_series = parsed_series.fillna(pd.to_datetime(series, errors='coerce'))

    return parsed_series

if 'INFODT' in motor_df.columns:
    motor_df['INFODT_parsed'] = robust_date_parse(motor_df['INFODT'], date_formats_to_try)
if 'INFODT' in mri_df.columns:
    mri_df['INFODT_parsed'] = robust_date_parse(mri_df['INFODT'], date_formats_to_try)

print("Date parsing attempted.")

Date parsing attempted.


/var/folders/kw/d3zgy6jx0gl0lybbg494yl8c0000gn/T/ipykernel_31633/1202056296.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed_series = parsed_series.fillna(pd.to_datetime(series, errors='coerce'))
/var/folders/kw/d3zgy6jx0gl0lybbg494yl8c0000gn/T/ipykernel_31633/1202056296.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed_series = parsed_series.fillna(pd.to_datetime(series, errors='coerce'))


In [5]:
print(f"\nMerging dataframes on: {merge_cols}")

# Using 'inner' merge: Keeps only rows where PATNO+EVENT_ID combination exists in BOTH datasets.
# Use 'left' merge if you want to keep ALL rows from the motor_df and add MRI data where it matches.
# Use suffixes to automatically rename overlapping columns (like INFODT, PAG_NAME, REC_ID, etc.)
# excluding the merge keys ('PATNO', 'EVENT_ID').
merged_df = pd.merge(
    motor_df,
    mri_df,
    on=merge_cols,
    how='inner', # Change to 'left' to keep all motor records, 'outer' for all records from both
    suffixes=('_motor', '_mri') # Appends suffix to overlapping column names
)

print(f"\nMerge complete. Resulting dataframe shape: {merged_df.shape}")

# --- Post-Merge Inspection ---
if merged_df.empty:
    print("Warning: The merged dataframe is empty. This might indicate no matching PATNO+EVENT_ID pairs "
          "based on the 'inner' merge strategy. Consider trying 'left' or 'outer' merge "
          "or double-check the merge key values in your CSV files.")
else:
    print("\n--- Merged Data Sample (First 5 Rows) ---")
    print(merged_df.head())

    # Identify and list overlapping columns that received suffixes
    print("\n--- Overlapping Columns (Excluding Merge Keys) ---")
    motor_cols_set = set(motor_df.columns)
    mri_cols_set = set(mri_df.columns)
    overlapping_cols = (motor_cols_set.intersection(mri_cols_set)) - set(merge_cols)

    if overlapping_cols:
        print(f"Columns with suffixes added: {sorted(list(overlapping_cols))}")
        # Example: Display the original and parsed date columns side-by-side if they overlapped
        if 'INFODT' in overlapping_cols:
             print("\nComparison of INFODT columns (example):")
             print(merged_df[[f'INFODT_motor', f'INFODT_mri']].head())
             if 'INFODT_parsed_motor' in merged_df.columns and 'INFODT_parsed_mri' in merged_df.columns:
                   print("\nComparison of INFODT_parsed columns (example):")
                   print(merged_df[[f'INFODT_parsed_motor', f'INFODT_parsed_mri']].head())
    else:
        print("No overlapping columns found besides merge keys.")

    # --- Save Merged Data ---
    print(f"\nSaving merged data to: {output_file}")
    try:
        merged_df.to_csv(output_file, index=False, encoding='utf-8')
        print(f"Successfully saved merged data to {output_file}")
    except Exception as e:
        print(f"Error saving merged data: {e}")


Merging dataframes on: ['PATNO', 'EVENT_ID']

Merge complete. Resulting dataframe shape: (8131, 78)

--- Merged Data Sample (First 5 Rows) ---
  REC_ID_motor  PATNO EVENT_ID PAG_NAME_motor INFODT_motor  PDTRTMNT PDSTATE  \
0    272451901   3000       BL        NUPDRS3      02/2011       NaN     NaN   
1    563731101   3000      V12        NUPDRS3      04/2016       NaN     NaN   
2    278743601   3001       BL        NUPDRS3      03/2011       0.0     NaN   
3    281541201   3002       BL        NUPDRS3      03/2011       0.0     NaN   
4    287051101   3003       BL        NUPDRS3      04/2011       0.0     NaN   

   HRPOSTMED  HRDBSON  HRDBSOFF  ...  MRICMPLT  MRIWDTI  MRIWRSS  MRIRSLT  \
0        NaN      NaN       NaN  ...       1.0      0.0      NaN      2.0   
1        NaN      NaN       NaN  ...       0.0      NaN      NaN      NaN   
2        NaN      NaN       NaN  ...       1.0      0.0      NaN      2.0   
3        NaN      NaN       NaN  ...       1.0      0.0      NaN   